# Low light Image enhancement 
In this notebook, we illustrate the use of the Zero-dce model for low light image enhancement using custom postprocessor with DeGirum PySDK

### Define custom postprocessor

In [ ]:

from __future__ import annotations

import numpy as np
import cv2
import degirum as dg
from degirum.postprocessor import InferenceResults, _inference_result_type, log_wrap
from degirum._model_param_helpers import model_shape_get


class EnhancementPostprocessor(InferenceResults):
    """Post‑processor that converts a flattened RGB tensor `(1, H×W, 3)` produced
    by low‑light enhancement models into a displayable `uint8` BGR image.

    **image** property → returns the *original* input frame.  
    **image_overlay** → returns the enhanced frame, automatically resized to the
    original resolution so the standard SDK viewers display a “before / after”
    stacked view.

    After construction:
    * ``self._enhanced_image`` holds the processed frame (OpenCV format).
    * ``self._inference_results`` is replaced with a one‑liner metadata dict.
    """

    @log_wrap
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        try:
            raw_tensor = np.asarray(self._inference_results[0]["data"],
                                     dtype=np.float32)
        except Exception as e:  # pragma: no cover – developer mistake guard
            raise RuntimeError(
                "EnhancementPostprocessor expects inference_results[0]['data'] to "
                "contain the flat output tensor.  Didn't find it: " + str(e)
            ) from e

        raw_tensor = raw_tensor.squeeze(0)          # (H×W, 3)

        try:
            _, H, W, _ = model_shape_get(self._model_params, 0, 4)
        except Exception:
            H, W = 400, 600  # Zero‑DCE default

        if raw_tensor.shape[0] != H * W:
            raise ValueError(
                f"Flat tensor length {raw_tensor.shape[0]} does not match H×W "
                f"({H}×{W}={H*W}).  Check model output/shape assumptions."
            )

        img = raw_tensor.reshape(H, W, 3)
        self._enhanced_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        if hasattr(self._input_image, "shape"):
            orig_h, orig_w = self._input_image.shape[:2]
        else:  # PIL.Image
            orig_w, orig_h = self._input_image.size

        self._output_image = cv2.resize(self._enhanced_image, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR)

    @property
    def image(self):
        """Return the *original* input frame unchanged."""
        return self._input_image
    
    def output_image(self):
        """Return the enhanced frame, resized to input resolution."""
        return self._output_image

    @property
    def image_overlay(self):
        """ Return the result frame with output overlay, resized to input resolution."""  
        return self._output_image


In [ ]:
import degirum as dg
import cv2

inference_host_address = "@cloud"
zoo_url = 'degirum/hailo'
token='dg_HVW6WKDH2YzmEonMbMibNJKkZJkXpKsLyHRdE'
device_type='HAILORT/HAILO8L'
model_name = 'zero_dce--400x600_quant_hailort_hailo8l_1'

model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type,
    custom_postprocessor= EnhancementPostprocessor,
)



# Load an image
enhanced_result = model('../assets/dark_room.jpg')

# Save image overlay output
cv2.imwrite("enhanced_image.jpg", enhanced_result.output_image())
print("Enhanced image saved as 'enhanced_image.jpg'")